# MAT 258A Final Project
## Dmitry Shemetov

We explore optimization methods in the parameter estimation of Hidden Markov models. 

The bottom line is as follows. Suppose we are given some discrete-time, finite-alphabet data $Y = [1,2,1,2,1,2,2,1]$. We seek parameters $\theta$ that maximize the likelihood of a particular HMM generating this data? Stated as an optimization problem:

$$\max_\theta L(\theta; Y) = P(Y| \theta)$$

Assuming a vector of hidden variables $X$, we have

$$P(Y | \theta) = \sum_X P(X,Y| \theta) = \sum_X P(X_1 = x_1) \prod P(X_{t+1}=x_{t+1} | X_t = x_t) P(Y_t=y_t| X_t = x_t, X_{t+1}=x_{t+1} ) $$

$$= \sum_X \pi_{X_1} \prod_{t=1}^T A_{X_t,X_{t+1}} B_{X_t,X_{t+1}}^{Y_t}$$

In the last line, we've encoded probabilities into three parameter sets: the starting distribution vector $\pi$, the hidden state transition matrix $A$ and the observable emission matrix $B$. These are all collected into a single parameter $\theta = \{ \pi, A, B \}$.

In [1]:
using Iterators
#using ReverseDiffSource

## 1. Parameter set-up.
Here we define the HMM processes. We use two: the even process and the coupled GMP process. Elsewhere, we have run a simulation using these processes and generated time-series. We will use this to try to reconstruct these ground truth parameters. We also define "guess" processes that are perturbations of these true processes. They will form our starting conditions for the iterative methods.

Even:
<img src="img/even.png">

and

Coupled Golden Mean Process:
<img src="img/cgmp.png">

In [2]:
# Even process.
pi = [1.0/3,2.0/3]
A = [0.5 0.5; 1.0 0]
B = randn(2,2,2)
B[:,:,1] = [1.0 0.0; 0.0 0.0]
B[:,:,2] = [0.0 1.0; 1.0 0.0]
even = {pi, A, B}

# Data
a = Int32[0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1]
evenY = [i+1 for i in a]

# Even guess.
pi = [1.0/10,9.0/10]
A = [0.1 0.9; 1.0 0]
B = randn(2,2,2)
B[:,:,1] = [1.0 0.0; 0.0 0.0]
B[:,:,2] = [0.0 1.0; 1.0 0.0]
evenGuess = {pi,A,B}


# Coupled GMP.
pi = [1.0/3, 1.0/6, 1.0/3, 1.0/6]
A = [0.5 0.5 0 0; 0.995 0.0 0.005 0.; 0 0 0.5 0.5; 0.005 0 0.995 0.]
B = randn(4,4,2)
B[:,:,1] = [0 1.0 0 0; 0 0 1.0 0; 0 0 1.0 0; 0 0 1.0 0]
B[:,:,2] = [1.0 0 0 0; 1.0 0 0 0; 0 0 0 1.0; 1.0 0 0 0]
cgmp = {pi, A, B}

# Data.
cgmpY = Int32[2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2]


# CGMP guess.
pi = [.5/3, .5/6, .5/3, 1.0/6]
A = [0.25 0.75 0 0; 0.5 0.0 0.5 0.; 0 0 0.25 0.75; 0.1 0 0.9 0.]
B = randn(4,4,2)
B[:,:,1] = [0 1.0 0 0; 0 0 1.0 0; 0 0 1.0 0; 0 0 1.0 0]
B[:,:,2] = [1.0 0 0 0; 1.0 0 0 0; 0 0 0 1.0; 1.0 0 0 0]
cgmpGuess = {pi, A, B};

## 2. Function definitions.
Here we define a number of functions. There are various helper functions, but most importantly are LL and gradLL, that are the loglikelihood and the gradient of the log-likelihood, respectively.

Investigating the log-likelihood function, we can note that there is a major barrier to computation: the summation over all the possible hidden state paths $X$. This makes the methods completely unusable when the length of the data strings goes beyond 20. 

In [3]:
# Log-Likelihood function for an HMM.
function LL(theta,Y)
    pi,A,B = theta[1], theta[2], theta[3]
    s = prob(theta,Y)
    return log(s)
end

# Gradient of log-likelihood.
function gradLL(theta,Y)
    pi,A,B = theta[1], theta[2], theta[3]
    n = length(A[1,:]) # Hidden states
    m = length(B[1,1,:]) # Observed states
    T = length(Y) # Data length
    xpaths = getpaths(n,T+1)
    
    dldpi = zeros(n)
    dldA = zeros(n,n)
    dldB = zeros(n,n,m)
    
    for i in 1:n
        for X in getpaths(n,T+1-1)
            if pi[i] != 0.0
                x = [i; [x1 for x1 in X]]
                dldpi[i] += 1/pi[i] * prob(theta,x,Y)
            end
        end
    end

    #=
    for i in 1:n
        for X in xpaths
            if X[1] == i && pi[i] != 0.0
                dldpi[i] += 1/pi[i] * prob(theta,X,Y)
            end
        end
    end
    =#
    
    for i in 1:n
        for j in 1:n
            for X in xpaths
                p = prob(theta,X,Y)
                for t in 1:T
                    if X[t]==i && X[t+1]==j && A[i,j] != 0.0
                        dldA[i,j] += 1/A[i,j] * p
                    end
                end
            end
        end
    end
    
    for i in 1:n
        for j in 1:n
            for k in 1:m
                for X in xpaths
                    p = prob(theta,X,Y)
                    for t in 1:T
                        if X[t]==i && X[t+1]==j && Y[t] == k && B[i,j,k] != 0.0
                            dldB[i,j,k] += 1/B[i,j,k] * p
                        end
                    end
                end
            end
        end
    end
    
    dldtheta = {dldpi, dldA, dldB}
    
    return dldtheta
end

# The expectation function
function Q(thetav,theta)
    piv,Av,Bv = thetav[1], thetav[2], thetav[3]
    pi,A,B = theta[1], theta[2], theta[3]
    n = length(A[1,:]) # Hidden states
    m = length(B[1,1,:]) # Observed states
    T = length(Y) # Data length
    xpaths = getpaths(n,T+1)
    
    s = 0
    for X in xpaths
        s += log(P(theta,X,Y)) * P(thetav,X,Y)
    end
    
    return s
end

function dQ(thetav,theta)
    piv,Av,Bv = thetav[1], thetav[2], thetav[3]
    pi,A,B = theta[1], theta[2], theta[3]
    n = length(A[1,:]) # Hidden states
    m = length(B[1,1,:]) # Observed states
    T = length(Y) # Data length
    xpaths = getpaths(n,T+1)
    
    dqdpi = zeros(n)
    dqdA = zeros(n,n)
    dqdB = zeros(n,n,m)
    
    for i in 1:n
        for X in getpaths(n,T+1-1)
            if pi[i] != 0.0
                x = [i; [x1 for x1 in X]]
                dqdpi[i] += 1/pi[i] * prob(thetav,x,Y)
            end
        end
    end

    #=
    for i in 1:n
        for X in xpaths
            if X[1] == i && pi[i] != 0.0
                dqdpi[i] += 1/pi[i] * prob(thetav,X,Y)
            end
        end
    end
    =#
    
    for i in 1:n
        for j in 1:n
            for X in xpaths
                p = prob(thetav,X,Y)
                for t in 1:T
                    if X[t]==i && X[t+1]==j && A[i,j] != 0.0
                        dqdA[i,j] += 1/A[i,j] * p
                    end
                end
            end
        end
    end
    
    for i in 1:n
        for j in 1:n
            for k in 1:m
                for X in xpaths
                    p = prob(thetav,X,Y)
                    for t in 1:T
                        if X[t]==i && X[t+1]==j && Y[t] == k && B[i,j,k] != 0.0
                            dqdB[i,j,k] += 1/B[i,j,k] * p
                        end
                    end
                end
            end
        end
    end
    
    dqdtheta = {dqdpi, dqdA, dqdB}
    
    return dqdtheta
end


# Computes P(Y | theta).
function prob(theta,Y)
    s = 0
    n = length(theta[2][1,:])
    T = length(theta[3][1,1,:])
    for X in getpaths(n,T+1)
        s += prob(theta,X,Y)
    end

    return s
end

# Computes P(X, Y| theta).
function prob(theta,X,Y)
    pi,A,B = theta[1], theta[2], theta[3]
    p = pi[X[1]]
    T = length(theta[3][1,1,:])
    for t in 1:T
        if p == 0
            break
        end
        p = p * A[X[t],X[t+1]] * B[X[t],X[t+1],Y[t]]
    end
    return p
end

# Objective function.
function obj(x,Y)
    f = LL(x,Y)
    g = gradLL(x,Y)
    gv = [g[1][:]; g[2][:]; g[3][:]]
    H = eye(3)
    return (f,g,gv,H)
end

# Reshape theta out of vector form.
function thetashaper(th,n,m)
    theta = {th[1:n], reshape(th[(n+1):(n + n^2)],n,n), reshape(th[(n+1+n^2):end],n,n,m)}
    return theta
end

# Create hidden state paths.
# A convoluted metaprogramming route to taking the n^th Cartesian product of
# the vector 1:T. Really, Julia?
function getpaths(n,T)
    cmd = string("product(1:",n)
    ran = string(",1:",n)
    for i in 1:(T-1)
        cmd = string(cmd,ran)
    end
    cmd = string(cmd,")") # cmd should now be "product(1:(T-1),...,1:(T-1))"
    xpaths = eval(parse(cmd))
    return xpaths
end

1;

## 3.1 Direct optimization - steepest descent.
### 3.1.1 Even.
We now apply steepest descent to both the evenGuess and the cgmpGuess, in the hope that they converge to the original HMM that produced the data.

In [4]:
# Steepest descent.
thetac = evenGuess
its = 0
gv = [2]
T = 10
# Vars.
n = length(thetac[2][1,:]) # Hidden states
m = length(thetac[3][1,1,:]) # Observed states


tic();
while its < 10 && norm(gv,2) > 1.0e-2
    (f,g,gv,H) = obj(thetac,evenY[1:T])
    thetan = thetac + g

    # Normalize (project).
    thetan[1] /= sum(thetan[1])
    for i in 1:n
        if sum(thetan[2][i,:]) != 0
            thetan[2][i,:] /= sum(thetan[2][i,:])
        end
    end
    for k in 1:m
        for i in 1:n
            if sum(thetan[3][i,:,k]) != 0
                thetan[3][i,:,k] /= sum(thetan[3][i,:,k])
            end
        end
    end
    
    its += 1
    thetac = thetan
end
tim = toc();

print("Running Dmitry's Steepest Descent method on the even process!\n\n")
@printf "%10s %20s %20s %20s\n" "Iterations" "Time" "Norm(grad)" "Norm(true - found)"
tr = [even[1][:]; even[2][:]; even[3][:]]
@printf "%10d %20f %20f %20f\n" its norm(gv,2) tim norm(tr[3:end] - gv[3:end],2)

elapsed time: 3.914792493 seconds
Running Dmitry's Steepest Descent method on the even process!

Iterations                 Time           Norm(grad)   Norm(true - found)
        10          3463.939592             3.914792          3457.600059


### Inferred parameters:

In [5]:
thetac

3-element Array{Any,1}:
 [1.0,3.195351973838682e-17]                                                             
 2x2 Array{Float64,2}:
 0.225097  0.774903
 1.0       0.0                                
 2x2x2 Array{Float64,3}:
[:, :, 1] =
 1.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  1.0
 1.0  0.0

In [6]:
even

3-element Array{Any,1}:
 [0.3333333333333333,0.6666666666666666]                                                 
 2x2 Array{Float64,2}:
 0.5  0.5
 1.0  0.0                                               
 2x2x2 Array{Float64,3}:
[:, :, 1] =
 1.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  1.0
 1.0  0.0

### 3.1.2 CGMP.
Now we run the same procedure on the cgmp process.

In [7]:
# Steepest descent.
thetac = cgmpGuess
its = 0
gv = [2]
T = 6
# Vars.
n = length(thetac[2][1,:]) # Hidden states
m = length(thetac[3][1,1,:]) # Observed states


tic();
while its < 2 && norm(gv,2) > 1.0e-2
    (f,g,gv,H) = obj(thetac,cgmpY[1:T])
    thetan = thetac + g

    # Normalize (project).
    thetan[1] /= sum(thetan[1])
    for i in 1:n
        if sum(thetan[2][i,:]) != 0
            thetan[2][i,:] /= sum(thetan[2][i,:])
        end
    end
    for k in 1:m
        for i in 1:n
            if sum(thetan[3][i,:,k]) != 0
                thetan[3][i,:,k] /= sum(thetan[3][i,:,k])
            end
        end
    end
    
    its += 1
    thetac = thetan
end
tim = toc();

print("Running Dmitry's Steepest Descent method on the even process!\n\n")
@printf "%10s %20s %20s %20s\n" "Iterations" "Time" "Norm(grad)" "Norm(true - found)"
tr = [cgmp[1][:]; cgmp[2][:]; cgmp[3][:]]
@printf "%10d %20f %20f %20f\n" its norm(gv,2) tim norm(tr[n+1:end] - gv[n+1:end],2)

elapsed time: 14.958509811 seconds
Running Dmitry's Steepest Descent method on the even process!

Iterations                 Time           Norm(grad)   Norm(true - found)
         2          1120.991065            14.958510          1059.777048


### Inferred parameters:

In [8]:
thetac

3-element Array{Any,1}:
 [0.29021386187832066,0.22675711568665907,0.17324892604453904,0.30978009639048126]                                                                                                                               
 4x4 Array{Float64,2}:
 0.292596  0.707404  0.0       0.0     
 0.584736  0.0       0.415264  0.0     
 0.0       0.0       0.329875  0.670125
 0.113624  0.0       0.886376  0.0                                
 4x4x2 Array{Float64,3}:
[:, :, 1] =
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0

[:, :, 2] =
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0

In [9]:
cgmp

3-element Array{Any,1}:
 [0.3333333333333333,0.16666666666666666,0.3333333333333333,0.16666666666666666]                                                                                                                                 
 4x4 Array{Float64,2}:
 0.5    0.5  0.0    0.0
 0.995  0.0  0.005  0.0
 0.0    0.0  0.5    0.5
 0.005  0.0  0.995  0.0                                                                                           
 4x4x2 Array{Float64,3}:
[:, :, 1] =
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0

[:, :, 2] =
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0

## 3.2 Direct optimization - BFGS.
### 3.2.1 Even.

In [10]:
function thetaunshaper(theta)
    return [theta[1][:]; theta[2][:]; theta[3][:]]
end

thetaunshaper (generic function with 1 method)

In [11]:
# BFGS.
# Initialize parameters.
its = 0
thetac = evenGuess
n = length(thetac[2][1,:]) # Hidden states
m = length(thetac[3][1,1,:]) # Observed states
T = 10

(f,g,gv,H) = obj(thetac,evenY[1:T])
B = eye(length(gv))
d = []
tic();
while (its < 4 && norm(gv,2)>1.0e-2)
    (f,g,gv,H) = obj(thetac,evenY[1:T])
    gv = [g[1][:], g[2][:], g[3][:]]
    d = B \ (-gv)

    # Backtracking linesearch.
    alpha = .01;
    mu = 10^-1.0;
    th = thetaunshaper(thetac)
    (newf,newg,newgv,_) = obj(thetashaper(abs(th+alpha*d),n,m),evenY[1:T]);
    while newf > f + (alpha*mu)*(dot(gv,d))
        (newf,newg,newgv,_) = obj(thetashaper(abs(th+alpha*d),n,m),evenY[1:T]);
        alpha = alpha/2;
    end
    
    # Update x.
    thetan = thetac + thetashaper(alpha*d,n,m)
    thetan[1] = abs(thetan[1])
    thetan[2] = abs(thetan[2])
    thetan[3] = abs(thetan[3])
    
    # Normalize.
    thetan[1] /= sum(thetan[1])
    for i in 1:n
        if sum(thetan[2][i,:]) != 0
            thetan[2][i,:] /= sum(thetan[2][i,:])
        end
    end
    for k in 1:m
        for i in 1:n
            if sum(thetan[3][i,:,k]) != 0
                thetan[3][i,:,k] /= sum(thetan[3][i,:,k])
            end
        end
    end
    
    thetac = thetan
    
    
    # Perform the BFGS update.
    s = alpha*d
    y = newgv - gv
    B = B + (y*y') / dot(y,s) - ((B*s)*(s'*B)) / dot(s,B*s)

    # Increment and store data.
    its += 1
end
tim = toc();

print("Running Dmitry's BFGS method on the even process!\n\n")
@printf "%10s %20s %20s %20s\n" "Iterations" "Time" "Norm(grad)" "Norm(true - found)"
tr = thetaunshaper(even)
@printf "%10d %20f %20f %20f\n" its norm(gv,2) tim norm(tr[n+1:end] - gv[n+1:end],2)

 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at abstractarray.jl:29
 in vect at abstractarray.jl:32
 [inlined code] from In[11]:15
 in anonymous at no file:14
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at C:\Users\Dmitry\.julia\v0.4\IJulia\src\execute_request.jl:177
 in eventloop at C:\Users\Dmitry\.julia\v0.4\IJulia\src\IJulia.jl:141
 in anonymous at task.jl:447
while loading In[11], in expression starting on line 13

elapsed time: 


 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at abstractarray.jl:29
 in vect at abstractarray.jl:32
 [inlined code] from In[11]:15
 in anonymous at no file:14
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at C:\Users\Dmitry\.julia\v0.4\IJulia\src\execute_request.jl:177
 in eventloop at C:\Users\Dmitry\.julia\v0.4\IJulia\src\IJulia.jl:141
 in anonymous at task.jl:447
while loading In[11], in expression starting on line 13


4.46831473 seconds
Running Dmitry's BFGS method on the even process!

Iterations                 Time           Norm(grad)   Norm(true - found)
         4             0.766818             4.468315             2.072375


In [12]:
thetac

3-element Array{Any,1}:
 [0.07935357985067698,0.9206464201493231]                                                
 2x2 Array{Float64,2}:
 0.00874852  0.991251
 1.0         0.0                            
 2x2x2 Array{Float64,3}:
[:, :, 1] =
 1.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  1.0
 1.0  0.0

In [13]:
even

3-element Array{Any,1}:
 [0.3333333333333333,0.6666666666666666]                                                 
 2x2 Array{Float64,2}:
 0.5  0.5
 1.0  0.0                                               
 2x2x2 Array{Float64,3}:
[:, :, 1] =
 1.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  1.0
 1.0  0.0

### 3.2.2 CGMP

In [14]:
# BFGS.
# Initialize parameters.
its = 0
thetac = cgmpGuess
n = length(thetac[2][1,:]) # Hidden states
m = length(thetac[3][1,1,:]) # Observed states
T = 5

(f,g,gv,H) = obj(thetac,cgmpY[1:T])
B = eye(length(gv))
d = []
tic();
while (its < 4 && norm(gv,2)>1.0e-2)
    (f,g,gv,H) = obj(thetac,cgmpY[1:T])
    gv = [g[1][:], g[2][:], g[3][:]]
    d = B \ (-gv)

    # Backtracking linesearch.
    alpha = .01;
    mu = 10^-1.0;
    th = thetaunshaper(thetac)
    (newf,newg,newgv,_) = obj(thetashaper(abs(th+alpha*d),n,m),cgmpY[1:T]);
    while newf > f + (alpha*mu)*(dot(gv,d))
        (newf,newg,newgv,_) = obj(thetashaper(abs(th+alpha*d),n,m),cgmpY[1:T]);
        alpha = alpha/2;
    end
    
    # Update x.
    thetan = thetac + thetashaper(alpha*d,n,m)
    thetan[1] = abs(thetan[1])
    thetan[2] = abs(thetan[2])
    thetan[3] = abs(thetan[3])
    
    # Normalize.
    thetan[1] /= sum(thetan[1])
    for i in 1:n
        if sum(thetan[2][i,:]) != 0
            thetan[2][i,:] /= sum(thetan[2][i,:])
        end
    end
    for k in 1:m
        for i in 1:n
            if sum(thetan[3][i,:,k]) != 0
                thetan[3][i,:,k] /= sum(thetan[3][i,:,k])
            end
        end
    end
    
    thetac = thetan
    
    
    # Perform the BFGS update.
    s = alpha*d
    y = newgv - gv
    B = B + (y*y') / dot(y,s) - ((B*s)*(s'*B)) / dot(s,B*s)

    # Increment and store data.
    its += 1
end
tim = toc();

print("Running Dmitry's BFGS method on the even process!\n\n")
@printf "%10s %20s %20s %20s\n" "Iterations" "Time" "Norm(grad)" "Norm(true - found)"
tr = thetaunshaper(cgmp)
@printf "%10d %20f %20f %20f\n" its norm(gv,2) tim norm(tr[n+1:end] - gv[n+1:end],2)

elapsed time: 15.325888577 seconds
Running Dmitry's BFGS method on the even process!

Iterations                 Time           Norm(grad)   Norm(true - found)
         4            23.687618            15.325889            23.185248


### Inferred parameters

In [15]:
thetac

3-element Array{Any,1}:
 [0.34605151222912617,0.029073980886746335,0.256393108183601,0.3684813987005266]                                                                                                                                 
 4x4 Array{Float64,2}:
 0.076973   0.923027  0.0       0.0     
 0.541069   0.0       0.458931  0.0     
 0.0        0.0       0.259383  0.740617
 0.0158367  0.0       0.984163  0.0                            
 4x4x2 Array{Float64,3}:
[:, :, 1] =
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0

[:, :, 2] =
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0

In [16]:
cgmp

3-element Array{Any,1}:
 [0.3333333333333333,0.16666666666666666,0.3333333333333333,0.16666666666666666]                                                                                                                                 
 4x4 Array{Float64,2}:
 0.5    0.5  0.0    0.0
 0.995  0.0  0.005  0.0
 0.0    0.0  0.5    0.5
 0.005  0.0  0.995  0.0                                                                                           
 4x4x2 Array{Float64,3}:
[:, :, 1] =
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0

[:, :, 2] =
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0

## 4.1 Expectation maximization - steepest descent.
### 4.1.1 Even.

In [17]:
function obj(theta,thetav,Y)
    f = q(theta,thetav)
    g = dq(theta,thetav)
    gv = thetaunshaper(g)
    n = length(theta[1][1,:])
    H = ones(n)
    return (f,g,gv,H)
end

obj (generic function with 2 methods)

In [18]:
# Steepest descent.
thetac = evenGuess
its = 0
gv = [2]
T = 10
# Vars.
n = length(thetac[2][1,:]) # Hidden states
m = length(thetac[3][1,1,:]) # Observed states


tic();
while its < 10 && norm(gv,2) > 1.0e-2
    (f,g,gv,H) = obj(thetac,evenY[1:T])
    thetan = thetac + g

    # Normalize (project).
    thetan[1] /= sum(thetan[1])
    for i in 1:n
        if sum(thetan[2][i,:]) != 0
            thetan[2][i,:] /= sum(thetan[2][i,:])
        end
    end
    for k in 1:m
        for i in 1:n
            if sum(thetan[3][i,:,k]) != 0
                thetan[3][i,:,k] /= sum(thetan[3][i,:,k])
            end
        end
    end
    
    its += 1
    thetac = thetan
end
tim = toc();

print("Running Dmitry's Steepest Descent method on the even process!\n\n")
@printf "%10s %20s %20s %20s\n" "Iterations" "Time" "Norm(grad)" "Norm(true - found)"
tr = [even[1][:]; even[2][:]; even[3][:]]
@printf "%10d %20f %20f %20f\n" its norm(gv,2) tim norm(tr[3:end] - gv[3:end],2)

elapsed time: 2.567484011 seconds
Running Dmitry's Steepest Descent method on the even process!

Iterations                 Time           Norm(grad)   Norm(true - found)
        10          3463.939592             2.567484          3457.600059


### Inferred parameters

In [19]:
thetac

3-element Array{Any,1}:
 [1.0,3.195351973838682e-17]                                                             
 2x2 Array{Float64,2}:
 0.225097  0.774903
 1.0       0.0                                
 2x2x2 Array{Float64,3}:
[:, :, 1] =
 1.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  1.0
 1.0  0.0

In [20]:
even

3-element Array{Any,1}:
 [0.3333333333333333,0.6666666666666666]                                                 
 2x2 Array{Float64,2}:
 0.5  0.5
 1.0  0.0                                               
 2x2x2 Array{Float64,3}:
[:, :, 1] =
 1.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  1.0
 1.0  0.0

### 4.1.2 CGMP.

In [21]:
# Steepest descent.
thetac = cgmpGuess
its = 0
gv = [2]
T = 6
# Vars.
n = length(thetac[2][1,:]) # Hidden states
m = length(thetac[3][1,1,:]) # Observed states


tic();
while its < 2 && norm(gv,2) > 1.0e-2
    (f,g,gv,H) = obj(thetac,cgmpY[1:T])
    thetan = thetac + g

    # Normalize (project).
    thetan[1] /= sum(thetan[1])
    for i in 1:n
        if sum(thetan[2][i,:]) != 0
            thetan[2][i,:] /= sum(thetan[2][i,:])
        end
    end
    for k in 1:m
        for i in 1:n
            if sum(thetan[3][i,:,k]) != 0
                thetan[3][i,:,k] /= sum(thetan[3][i,:,k])
            end
        end
    end
    
    its += 1
    thetac = thetan
end
tim = toc();

print("Running Dmitry's Steepest Descent method on the even process!\n\n")
@printf "%10s %20s %20s %20s\n" "Iterations" "Time" "Norm(grad)" "Norm(true - found)"
tr = [cgmp[1][:]; cgmp[2][:]; cgmp[3][:]]
@printf "%10d %20f %20f %20f\n" its norm(gv,2) tim norm(tr[n+1:end] - gv[n+1:end],2)

elapsed time: 11.428779227 seconds
Running Dmitry's Steepest Descent method on the even process!

Iterations                 Time           Norm(grad)   Norm(true - found)
         2          1120.991065            11.428779          1059.777048


### Inferred parameters.

In [22]:
thetac

3-element Array{Any,1}:
 [0.29021386187832066,0.22675711568665907,0.17324892604453904,0.30978009639048126]                                                                                                                               
 4x4 Array{Float64,2}:
 0.292596  0.707404  0.0       0.0     
 0.584736  0.0       0.415264  0.0     
 0.0       0.0       0.329875  0.670125
 0.113624  0.0       0.886376  0.0                                
 4x4x2 Array{Float64,3}:
[:, :, 1] =
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0

[:, :, 2] =
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0

In [23]:
cgmp

3-element Array{Any,1}:
 [0.3333333333333333,0.16666666666666666,0.3333333333333333,0.16666666666666666]                                                                                                                                 
 4x4 Array{Float64,2}:
 0.5    0.5  0.0    0.0
 0.995  0.0  0.005  0.0
 0.0    0.0  0.5    0.5
 0.005  0.0  0.995  0.0                                                                                           
 4x4x2 Array{Float64,3}:
[:, :, 1] =
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0

[:, :, 2] =
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0

## 4.2 Expectation maximization - BFGS.
### 4.2.1 Even.

In [24]:
# BFGS.
# Initialize parameters.
its = 0
thetac = evenGuess
n = length(thetac[2][1,:]) # Hidden states
m = length(thetac[3][1,1,:]) # Observed states
T = 10

(f,g,gv,H) = obj(thetac,evenY[1:T])
B = eye(length(gv))
d = []
tic();
while (its < 4 && norm(gv,2)>1.0e-2)
    (f,g,gv,H) = obj(thetac,evenY[1:T])
    gv = [g[1][:], g[2][:], g[3][:]]
    d = B \ (-gv)

    # Backtracking linesearch.
    alpha = .01;
    mu = 10^-1.0;
    th = thetaunshaper(thetac)
    (newf,newg,newgv,_) = obj(thetashaper(abs(th+alpha*d),n,m),evenY[1:T]);
    while newf > f + (alpha*mu)*(dot(gv,d))
        (newf,newg,newgv,_) = obj(thetashaper(abs(th+alpha*d),n,m),evenY[1:T]);
        alpha = alpha/2;
    end
    
    # Update x.
    thetan = thetac + thetashaper(alpha*d,n,m)
    thetan[1] = abs(thetan[1])
    thetan[2] = abs(thetan[2])
    thetan[3] = abs(thetan[3])
    
    # Normalize.
    thetan[1] /= sum(thetan[1])
    for i in 1:n
        if sum(thetan[2][i,:]) != 0
            thetan[2][i,:] /= sum(thetan[2][i,:])
        end
    end
    for k in 1:m
        for i in 1:n
            if sum(thetan[3][i,:,k]) != 0
                thetan[3][i,:,k] /= sum(thetan[3][i,:,k])
            end
        end
    end
    
    thetac = thetan
    
    
    # Perform the BFGS update.
    s = alpha*d
    y = newgv - gv
    B = B + (y*y') / dot(y,s) - ((B*s)*(s'*B)) / dot(s,B*s)

    # Increment and store data.
    its += 1
end
tim = toc();

print("Running Dmitry's BFGS method on the even process!\n\n")
@printf "%10s %20s %20s %20s\n" "Iterations" "Time" "Norm(grad)" "Norm(true - found)"
tr = thetaunshaper(even)
@printf "%10d %20f %20f %20f\n" its norm(gv,2) tim norm(tr[n+1:end] - gv[n+1:end],2)

elapsed time: 2.131270447 seconds
Running Dmitry's BFGS method on the even process!

Iterations                 Time           Norm(grad)   Norm(true - found)
         4             0.766818             2.131270             2.072375


### Inferred parameters.

In [25]:
thetac

3-element Array{Any,1}:
 [0.07935357985067698,0.9206464201493231]                                                
 2x2 Array{Float64,2}:
 0.00874852  0.991251
 1.0         0.0                            
 2x2x2 Array{Float64,3}:
[:, :, 1] =
 1.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  1.0
 1.0  0.0

In [27]:
even

3-element Array{Any,1}:
 [0.3333333333333333,0.6666666666666666]                                                 
 2x2 Array{Float64,2}:
 0.5  0.5
 1.0  0.0                                               
 2x2x2 Array{Float64,3}:
[:, :, 1] =
 1.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  1.0
 1.0  0.0

### 4.2.2 CGMP.

In [28]:
# BFGS.
# Initialize parameters.
its = 0
thetac = cgmpGuess
n = length(thetac[2][1,:]) # Hidden states
m = length(thetac[3][1,1,:]) # Observed states
T = 5

(f,g,gv,H) = obj(thetac,cgmpY[1:T])
B = eye(length(gv))
d = []
tic();
while (its < 4 && norm(gv,2)>1.0e-2)
    (f,g,gv,H) = obj(thetac,cgmpY[1:T])
    gv = [g[1][:], g[2][:], g[3][:]]
    d = B \ (-gv)

    # Backtracking linesearch.
    alpha = .01;
    mu = 10^-1.0;
    th = thetaunshaper(thetac)
    (newf,newg,newgv,_) = obj(thetashaper(abs(th+alpha*d),n,m),cgmpY[1:T]);
    while newf > f + (alpha*mu)*(dot(gv,d))
        (newf,newg,newgv,_) = obj(thetashaper(abs(th+alpha*d),n,m),cgmpY[1:T]);
        alpha = alpha/2;
    end
    
    # Update x.
    thetan = thetac + thetashaper(alpha*d,n,m)
    thetan[1] = abs(thetan[1])
    thetan[2] = abs(thetan[2])
    thetan[3] = abs(thetan[3])
    
    # Normalize.
    thetan[1] /= sum(thetan[1])
    for i in 1:n
        if sum(thetan[2][i,:]) != 0
            thetan[2][i,:] /= sum(thetan[2][i,:])
        end
    end
    for k in 1:m
        for i in 1:n
            if sum(thetan[3][i,:,k]) != 0
                thetan[3][i,:,k] /= sum(thetan[3][i,:,k])
            end
        end
    end
    
    thetac = thetan
    
    
    # Perform the BFGS update.
    s = alpha*d
    y = newgv - gv
    B = B + (y*y') / dot(y,s) - ((B*s)*(s'*B)) / dot(s,B*s)

    # Increment and store data.
    its += 1
end
tim = toc();

print("Running Dmitry's BFGS method on the even process!\n\n")
@printf "%10s %20s %20s %20s\n" "Iterations" "Time" "Norm(grad)" "Norm(true - found)"
tr = thetaunshaper(cgmp)
@printf "%10d %20f %20f %20f\n" its norm(gv,2) tim norm(tr[n+1:end] - gv[n+1:end],2)

elapsed time: 15.459707581 seconds
Running Dmitry's BFGS method on the even process!

Iterations                 Time           Norm(grad)   Norm(true - found)
         4            23.687618            15.459708            23.185248


### Inferred parameters.

In [29]:
thetac

3-element Array{Any,1}:
 [0.34605151222912617,0.029073980886746335,0.256393108183601,0.3684813987005266]                                                                                                                                 
 4x4 Array{Float64,2}:
 0.076973   0.923027  0.0       0.0     
 0.541069   0.0       0.458931  0.0     
 0.0        0.0       0.259383  0.740617
 0.0158367  0.0       0.984163  0.0                            
 4x4x2 Array{Float64,3}:
[:, :, 1] =
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0

[:, :, 2] =
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0

In [30]:
cgmp

3-element Array{Any,1}:
 [0.3333333333333333,0.16666666666666666,0.3333333333333333,0.16666666666666666]                                                                                                                                 
 4x4 Array{Float64,2}:
 0.5    0.5  0.0    0.0
 0.995  0.0  0.005  0.0
 0.0    0.0  0.5    0.5
 0.005  0.0  0.995  0.0                                                                                           
 4x4x2 Array{Float64,3}:
[:, :, 1] =
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0

[:, :, 2] =
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0